# Vishwas News

In [1]:
from selenium import webdriver
import time
import requests 
from bs4 import BeautifulSoup 
import pandas as pd
import re

In [2]:
# all categories 
url='https://www.vishvasnews.com/'
categories=['politics/','world/','viral/','society/','health/','videos/']
news_length=[149,872,796,383,127,59]

In [3]:
# links parsing from noise 
def parse_link(links_with_noise):
    links_=[link for noise in links_with_noise for link in str(noise).split('="') if link.startswith('https') and not link.endswith('</div>')]
    links=[l for link in links_ for l in link.split('">') if l.startswith('https')]
    return links

# Retriving all urls 

In [4]:
from selenium.webdriver.common.action_chains import ActionChains
driver = webdriver.Chrome("/home/shiva/python_projects_directory/chromedriver")
links=[]
for cat,l in zip(categories,news_length):
    driver.get(url+cat)
    for i in range(30):
        try:
            accept=driver.find_element_by_xpath("/html/body/div[8]/button")
            Hover = ActionChains(driver).move_to_element(accept)
            Hover.click().perform()
        except:
            pass
    count=0
    while count<=400:
        try:
            element=driver.find_element_by_partial_link_text('Load More')
            Hover = ActionChains(driver).move_to_element(element)
            Hover.click().perform()
        except:
            pass
        count+=1
        print(count)
    soup=BeautifulSoup(driver.page_source, 'lxml')
    links_with_noise = soup.findAll('div',class_='imagebox',attrs={'href'})
    links.extend(parse_link(links_with_noise))
f=open('vishwas_url.txt','w+')
for i in links:
    f.write(str(i)+'\n')
f.close()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=55827): Max retries exceeded with url: /session/6765ae429ba8c77c91bcbf9dd077bb0b/source (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f5fdaf16f98>: Failed to establish a new connection: [Errno 111] Connection refused',))

In [5]:
#  multple fake text concatenation  
# def con(lst):
#     strtxt=""
#     for txt in lst:
#         strtxt+=txt
#     return strtxt
# retrieve fake text which is under quotes only 
def txt_reterive(text):
    if re.findall('“.*',text) !=-1:
        text=' '.join([t.split('”')[0] for t in re.findall('“.*',text)])
        return text
    else:
        return ''
def archive_link(table):
    try:
        link=[i for i in table if len(re.findall('आर्काइव',str(i)))>0 ][-1]
        link=str(link).split('href="')[-1]
        link=re.sub('">लिंक.*','',link)
        return link
    except:
        return None

In [31]:
df=pd.DataFrame(columns=['title','link','date','text','archive','category'])
count=0
# accessing all links from file and access fake text,date title etc regarding that link 
with open('vishwas_url.txt','r') as f:
    for url in f.read().split('\n'):
        try:
            r = requests.get(url) 
            soup = BeautifulSoup(r.content, 'html.parser') 
            date=soup.find("ul",class_="updated",attrs={'li'})
            title=soup.find('h1',class_="article-heading")
            all_text = soup.find_all('p')
            archive=archive_link(all_text)
            # making dataframe 
            for txt in all_text:
                txt=txt_reterive(txt.text)
                if len(txt)>50:
                    df.loc[count]=[title.text,url,date.text,txt,archive,'fake']
                    print("Successfully Wrote row  {} after exteraction ".format(count))
                    count+=1      
        except Exception as e:
            print('something went wrong !!!',e)
            pass
# df.head()
# df.to_csv('vishwasNews_retrived.csv')
df=pd.read_csv('vishwasNews_retrived.csv')        

Successfully Wrote row  0 after exteraction 
Successfully Wrote row  1 after exteraction 
Successfully Wrote row  2 after exteraction 
Successfully Wrote row  3 after exteraction 
Successfully Wrote row  4 after exteraction 
Successfully Wrote row  5 after exteraction 
Successfully Wrote row  6 after exteraction 


KeyboardInterrupt: 

In [41]:
# df=pd.read_csv('vishwasNews_retrived.csv')
# df.date=df.date.apply(lambda x:x.split(':')[1].replace('\n','').strip())
# df.title=df.title.apply(lambda x:re.sub('[\n\r]','',x))
# df.to_csv('vishwasNews_retrived.csv')

,Unnamed: 0,Unnamed: 0.1,title,link,date,text,archive,category
0,0,0,Fact Check: MDMK प्रमुख वाइको अपने भती...,https://www.vishvasnews.com/politics/mdmk-chie...,"July 22, 2020",“एमडीएमके प्रमुख वाइको ने कोयंबटूर के सुंदरपुर...,"http://archive.fo/F3hor"">यहां </a>देखा जा सकता...",fake
1,1,1,Fact Check: MDMK प्रमुख वाइको अपने भती...,https://www.vishvasnews.com/politics/mdmk-chie...,"July 22, 2020",“यह वीडियो किसी हालिया घटना का नहीं है। वाइको ...,"http://archive.fo/F3hor"">यहां </a>देखा जा सकता...",fake
2,2,2,Fact Check: पशु अधिकार कार्यकर्ता के स...,https://www.vishvasnews.com/politics/fact-chec...,"July 21, 2020","“The Hindu extremist group RS, to which Prime ...","http://archive.fo/wip/g1lD9"">यहाँ </a>देखा जा ...",fake
3,3,3,Fact Check: पशु अधिकार कार्यकर्ता के स...,https://www.vishvasnews.com/politics/fact-chec...,"July 21, 2020","“‘हिंदू चरमपंथी समूह आरएस, जिसमें प्रधानमंत्री...","http://archive.fo/wip/g1lD9"">यहाँ </a>देखा जा ...",fake
4,4,4,Fact Check: पशु अधिकार कार्यकर्ता के स...,https://www.vishvasnews.com/politics/fact-chec...,"July 21, 2020","“3 जुलाई, 2020 को दिल्ली के रानी बाग इलाके में...","http://archive.fo/wip/g1lD9"">यहाँ </a>देखा जा ...",fake


In [ ]:
# for i,txt in enumerate(df.date):
#     print(str(i)+' : ',txt)

In [3]:
# vishwas new data retreiver 

from selenium import webdriver
import time
import requests 
from bs4 import BeautifulSoup 
import pandas as pd
import re

# retrieve fake text which is under quotes only 
def txt_reterive(text):
    if re.findall('“.*',text) !=-1:
        text=' '.join([t.split('”')[0] for t in re.findall('“.*',text)])
        return text
    else:
        return ''
def archive_link(table):
    try:
        link=[i for i in table if len(re.findall('आर्काइव',str(i)))>0 ][-1]
        link=str(link).split('href="')[-1]
        link=re.sub('">लिंक.*','',link)
        return link
    except:
        return None
    
def parse_link(links_with_noise):
    links_=[link for noise in links_with_noise for link in str(noise).split('="') if link.startswith('https') and not link.endswith('</div>')]
    links=[l for link in links_ for l in link.split('">') if l.startswith('https')]
    return links

def retrieve_vishwas_newlinks():
    """
    just call this function will return all links which show 
    """
    url='https://www.vishvasnews.com/'
    categories=['politics/','world/','viral/','society/','health/','videos/']
#     driver = webdriver.Chrome("/home/shiva/python_projects_directory/chromedriver")
    links=[]
    for cat in categories:
        r = requests.get(url+cat) 
        soup = BeautifulSoup(r.content, 'html.parser')
        links_with_noise = soup.findAll('div',class_='imagebox')
        links.extend(parse_link(links_with_noise))
    return links

def retrieve_new_data():
    df=pd.DataFrame(columns=['title','link','date','text','category','archive'])
    counter=0
    links=retrieve_vishwas_newlinks()
    for url in links:
        try:
            r = requests.get(url) 
            soup = BeautifulSoup(r.content, 'html.parser') 
            all_text= soup.find_all('p')
            date=soup.find("ul",class_="updated")
            title=soup.find('h1',class_="article-heading")
            archive=archive_link(all_text)
            # making dataframe 
            for txt in all_text:
                txt=txt_reterive(txt.text)
                if len(txt)>50:
                    df.loc[count]=[title.text,url,date.text,txt,'fake',archive]
                    print("Successfully Wrote row  {} after exteraction ".format(count))
                    count+=1 
    #         print(counter)
        except:
            print('something went wrong !!!')
    return df
# cleaning each column of dataframe retrieved from site

def archive_clean(x):
    x=re.sub('">.*','',str(x))
    return x
def clean(df):
    df.archive=df.archive.apply(archive_clean)
    return df       

In [24]:
from vishwas import retrieve_new_data,clean
import gspread
from oauth2client.service_account import ServiceAccountCredentials
credentials=ServiceAccountCredentials.from_json_keyfile_name('/home/shiva/Desktop/digit-260210-4b0b17907d28.json')
gc=gspread.authorize(credentials)
sheet=gc.open('test').sheet1

df=clean(retrieve_new_data())
for i in df.values:
    try:
        sheet.append_row(list(i))
    except APIError:
        pass

In [39]:
# sheet.append_row()


156

In [18]:
def sum(a,b):
    return a+b
try:
    a=[1,2,3,4,5,6]
    loc=int(input('Enter index')) # print no. until you input index , which exist in list a else raise Exception 
    print(a.index(loc))
except Exception as e: # start sum function if got a exception 
    print(sum(int(input('a: ')),int(input('b: '))))

Enter index12
12
34
46
